# NEB and CINEB for Ammonia Flipping. 
* Hanyu Ma and William Schneider
* Acknowledgement: Jerry, Craig and Jeonghyun

In [ ]:
from ase import Atoms
from ase.build import molecule
import numpy as np
from vasp import Vasp
from ase.constraints import FixAtoms
from ase.visualize import view
from vasp.vasprc import VASPRC
from ase.io import read,write

In [ ]:
# We use debug queue to run this job so we don't need to wait
VASPRC['queue.q'] = '*@@debug_d12chas'
VASPRC['queue.nprocs'] = 24

In [ ]:
# Load NH3 molecule from database
NH3 = molecule('NH3')
# Set a periodic cell
NH3.set_cell((10,10,10),scale_atoms=False)
# Move the NH3 molecule to the center
NH3.positions -= NH3.positions[0] - [5,5,5]
# Fix N atom
constraint = FixAtoms(mask=[atom.symbol == 'N' for atom in NH3])
NH3.set_constraint(constraint)
view(NH3)

In [ ]:
calc = Vasp('initial',
             xc = 'pbe',
             encut = 400,
             ibrion = 2,
#              ediffg = -0.03, # Turn ediffg on when you need a reliable calculation.
             nsw = 50,
             atoms = NH3)
calc.get_potential_energy()
# calc.view()

In [ ]:
# Create the final state (FS) using the optimized initial state
FS = read('initial/CONTCAR')

# Flip ammonia. 
for i in [1,2,3]:
    FS[i].position[2] += 2*(FS[0].position[2] - FS[i].position[2])
view (FS)

In [ ]:
calc = Vasp('final',
             xc = 'pbe',
             encut = 400,
             ibrion = 2,
             nsw = 50,
             atoms = FS)
calc.get_potential_energy()

In [ ]:
from ase.neb import NEB
IS = read('initial/CONTCAR')
FS = read('final/CONTCAR')

# Insert 4 images between IS and FS
images = [IS]
images += [IS.copy() for i in range(4)]
images += [FS]

# Update the positions of the images by linear interpolation
neb = NEB(images)
neb.interpolate()

view(images)

In [ ]:
# Let's run a normal NEB

calc = Vasp('NH3-neb',
            xc='PBE',
            ibrion=1, 
            encut=400,
            nsw=90,
            spring=-5.0,
#             lclimb = True,
            atoms=images)
# calc.view()

In [ ]:
# After the job is done. 
# (1) Move the OUTCAR file in "initial" and OUTCAR file in "final" to "NH3-neb/00" and "NH3-neb/05", respectively.
# (2) Move nebvis.py to "NH3-neb".
# (3) Type "python nebvis.py" to see the results.

In [ ]:
# Let's run a climbing image NEB

calc = Vasp('NH3-cineb',
            xc='PBE',
            ibrion=1, 
            encut=400,
            nsw=90,
            spring=-5.0,
            lclimb = True,
            atoms=images)
# calc.view()

In [ ]:
# After the job is done. 
# (1) Move the OUTCAR file in "initial" and OUTCAR file in "final" to "NH3-cineb/00" and "NH3-cineb/05", respectively.
# (2) Move nebvis.py to "NH3-cineb".
# (3) Run Python nebvis.py to see the results.

In [ ]:
# What are the differences between neb and cineb?

In [ ]:
# A three-image CINEB. 
# Does this reach agreement with the 4-image cineb?
from ase.neb import NEB
IS = read('initial/CONTCAR')
FS = read('final/CONTCAR')

# Insert 4 images between IS and FS
images = [IS]
images += [IS.copy() for i in range(3)]
images += [FS]

# Update the positions of the images by linear interpolation
neb = NEB(images)
neb.interpolate()

# Let's run a climbing image NEB

calc = Vasp('NH3-cineb-3-image',
            xc='PBE',
            ibrion=1, 
            encut=400,
            nsw=90,
            spring=-5.0,
            lclimb = True,
            atoms=images)
calc.view()